In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
search_query="smartphones"

In [ ]:
base_url="https://www.amazon.in/s?k="

In [ ]:
url=base_url+search_query

In [ ]:
url

In [ ]:
header={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87 Safari/537.36'}

In [ ]:
search_response=requests.get(url,headers=header)

In [ ]:
search_response.status_code

A function to get the content of the page of required query

In [ ]:
cookie={} # insert request cookies within{}
def getAmazonSearch(search_query,page):
    url=base_url+search_query+'&pageNumber='+str(page)
    print(url)
    page=requests.get(url,cookies=cookie,headers=header)
    if page.status_code==200:
        return page
    else:
        return "Error"

A function to get the contents of individual product pages using 'data-asin' number (unique identification number)

In [ ]:
def Searchasin(asin):
    url="https://www.amazon.in/dp/"+asin
    print(url)
    page=requests.get(url,cookies=cookie,headers=header)
    if page.status_code==200:
        return page
    else:
        return "Error"

A function to pass on the link of 'see all reviews' and extract the content

In [ ]:
def Searchreviews(review_link):
    url="https://www.amazon.in"+review_link
    print(url)
    page=requests.get(url,cookies=cookie,headers=header)
    if page.status_code==200:
        return page
    else:
        return "Error"

First page product reviews extraction

In [ ]:
product_names=[]
response=getAmazonSearch('smartphones',1)
soup=BeautifulSoup(response.content)

for i in soup.findAll("span",{'class':'a-size-medium a-color-base a-text-normal'}): # the tag which is common for all the names of products
    product_names.append(i.text)

findall() is used to find all the tags with the mentioned attributes and values in a page

In [ ]:
product_names

In [ ]:
len(product_names)

The method of extracting data-asin numbers are similar to that of product names. Only the tag details have to be changed in findall()

In [ ]:
data_asin=[]
pages=8
for i in range(pages):
    response=getAmazonSearch('smartphones',i)
    soup=BeautifulSoup(response.content)
    for i in soup.findAll("div",{'class':"sg-col-20-of-24 s-result-item s-asin sg-col-0-of-12 sg-col-28-of-32 sg-col-16-of-20 sg-col sg-col-32-of-36 sg-col-12-of-16 sg-col-24-of-28"}):
        data_asin.append(i['data-asin'])
    for i in soup.findAll("div",{'class':"sg-col-20-of-24 s-result-item s-asin sg-col-0-of-12 sg-col-28-of-32 sg-col-16-of-20 AdHolder sg-col sg-col-32-of-36 sg-col-12-of-16 sg-col-24-of-28"}):
        data_asin.append(i['data-asin'])

In [ ]:
data_asin[:5]

In [ ]:
len(data_asin)

By passing the data-asin numbers, we can extract the 'see all reviews' link for each product in the page

In [ ]:
links=[]
mobile_names=[]
asins=[]
for i in range(len(data_asin)):
        response=Searchasin(data_asin[i])
        soup=BeautifulSoup(response.content)
        mobile=soup.find("span",{"id":"productTitle"})
        mobile_names.append(mobile.string)
        link= soup.findAll("a",{'data-hook':"see-all-reviews-link-foot"})
        links.append(link[0]['href'])
        asins.append(data_asin[i])

In [ ]:
print(len(links),len(mobile_names),len(asins))
mobile_names[80]

In [ ]:
reviews=[]
for j in range(len(asins)):
    for k in range(10):
        response=Searchreviews(links[j]+'&pageNumber='+str(k))
        soup=BeautifulSoup(response.content)
        titles=soup.findAll("a",{'data-hook':"review-title"})
        bodies=soup.findAll("span",{'data-hook':"review-body"})
        stars=soup.findAll("i",{'data-hook':"review-star-rating"})
        for i in range(len(titles)):
            reviews.append([mobile_names[j],asins[j],titles[i].text,  bodies[i].text,  int(stars[i].text[0])])  
print('Number of reviews:',len(reviews))

In [ ]:
review_data = pd.DataFrame(reviews, columns = ['mobile_names','asin','title', 'body','star'])  
print(review_data.shape)

In [ ]:
review_data.sample(5)

Saving the output in csv

In [ ]:
review_data.to_csv('raw_reviews.csv',index=False)